In [2]:
#q8
!hdfs dfsadmin -fs hdfs://boss:9000 -report | grep "Live datanodes"

Live datanodes (2):


In [6]:
#q9
import requests
from urllib.parse import urlparse

block_distribution = {}

for offset in range(0, 174944099, 2 ** 20):
    response = requests.get(f'http://boss:9870/webhdfs/v1/single.csv?op=OPEN&offset={offset}&noredirect=true')
    
    if response.status_code == 403:
        block_distribution['lost'] = block_distribution.get('lost', 0) + 1
        continue
    
    container_id = urlparse(response.json()['Location']).hostname
    block_distribution[container_id] = block_distribution.get(container_id, 0) + 1
        
block_distribution

{'lost': 92, '0b4a56935bb6': 75}

In [1]:
#q10

import requests
import pyarrow as pa
import pyarrow.fs

def get_good_blocks(filename):
    url = f"http://boss:9870/webhdfs/v1/{filename}?op=GET_BLOCK_LOCATIONS"
    response = requests.get(url)
    blocks = response.json()['LocatedBlocks']['locatedBlocks']
    good_blocks = []
    for block in blocks:
        if block['locations']:
            good_blocks.append({
                'offset': block['startOffset'],
                'length': block['block']['numBytes']
            })
    return good_blocks

def count_lines_in_good_blocks(filename, search_string):
    good_blocks = get_good_blocks(filename)
    count = 0
    hdfs = pa.fs.HadoopFileSystem(host="boss", port=9000)
    with hdfs.open_input_file(filename) as file:
        for block in good_blocks:
            file.seek(block['offset'])
            data = file.read(block['length']).decode('utf-8')
            count += data.count(search_string)
    return count

num_occurrences = count_lines_in_good_blocks("/single.csv", "Single Family")
num_occurrences

KeyError: 'LocatedBlocks'